<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/6.4.%20%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E7%9A%84%E4%BB%8E%E9%9B%B6%E5%BC%80%E5%A7%8B%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##6.4. 循环神经网络的从零开始实现
在本节中，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集：

In [0]:
%matplotlib inline
import math
import tensorflow as tf
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.data import Dataset
import time
import random
import zipfile

In [0]:
tf.enable_eager_execution()

In [0]:
def load_data_jay_lyrics():
  from google.colab import drive
  drive.mount('/content/drive')
  with zipfile.ZipFile('/content/drive/My Drive/data/d2l-zh-tensoflow/jaychou_lyrics.txt.zip')as zin:
    with zin.open('jaychou_lyrics.txt') as f:
      corpus_chars=f.read().decode('utf-8')
  corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
  corpus_chars=corpus_chars[0:10000]
  idx_to_char=list(set(corpus_chars))
  char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
  vocab_size=len(char_to_idx)
  corpus_indices=[char_to_idx[char] for char in corpus_chars]
  return corpus_indices,char_to_idx,idx_to_char,vocab_size

(corpus_indices,char_to_idx,idx_to_char,vocab_size)=load_data_jay_lyrics() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###6.4.1. one-hot向量
为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为 N （即词典大小vocab_size），每个字符已经同一个从0到 N−1 的连续整数值索引一一对应。如果一个字符的索引是整数 i , 那么我们创建一个全0的长为 N 的向量，并将其位置为 i 的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [0]:
tf.one_hot(indices=[0,2],depth=vocab_size)

<tf.Tensor: id=4, shape=(2, 1027), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步 t 的输入为 $X_t \in R_n \times d$ ，其中 $n$ 为批量大小， d 为输入个数，即one-hot向量长度（词典大小）。

In [0]:
def to_onehot(X,size):
  return [tf.one_hot(x,size) for x in tf.transpose(X)]

X=tf.reshape(tf.range(10),(2,5))
inputs=to_onehot(X,vocab_size)
len(inputs),inputs[0].shape

(5, TensorShape([Dimension(2), Dimension(1027)]))

###6.4.2. 初始化模型参数
接下来，我们初始化模型参数。隐藏单元个数 num_hiddens是一个超参数。

In [0]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

def get_params():
  def _one(shape):
    return tf.Variable(tf.random.normal(stddev=0.01,shape=shape))

  #隐藏层参数
  W_xh=_one((num_inputs,num_hiddens)) #输入和隐藏层
  W_hh=_one((num_hiddens,num_hiddens)) #隐藏层之间
  b_h=tf.Variable(tf.zeros(num_hiddens))

  #输出层参数
  W_hq=_one((num_hiddens,num_outputs)) #隐藏层和输出层
  b_q=tf.Variable(tf.zeros(num_outputs))
  params=[W_xh,W_hh,b_h,W_hq,b_q]
  return params

###6.4.3. 定义模型
我们根据循环神经网络的计算表达式实现该模型。首先定义init_rnn_state函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。使用元组是为了更便于处理隐藏状态含有多个NDArray的情况。

In [0]:
def init_rnn_state(batch_size,num_hiddens):
  return (tf.zeros(shape=(batch_size,num_hiddens)),)

下面的rnn函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。“多层感知机”一节中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [0]:
def rnn(inputs,state,params):
  #inputs和outputs皆为num_steps个形状为(batch_size,vocab_size)的矩阵
  W_xh,W_hh,b_h,W_hq,b_q=params
  H,=state
  outputs=[]
  for X in inputs:
    H=tf.tanh(tf.matmul(X,W_xh)+tf.matmul(H,W_hh)+b_h)
    Y=tf.matmul(H,W_hq)+b_q
    outputs.append(Y)

  return outputs,(H,)

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [0]:
state=init_rnn_state(X.shape[0],num_hiddens)
inputs=to_onehot(X,vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
len(outputs),outputs[0].shape,state_new[0].shape

(5,
 TensorShape([Dimension(2), Dimension(1027)]),
 TensorShape([Dimension(2), Dimension(256)]))

###6.4.4. 定义预测函数
以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。这个函数稍显复杂，其中我们将循环神经单元rnn设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [0]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,idx_to_char,char_to_idx):
  state=init_rnn_state(1,num_hiddens)
  output=[char_to_idx[prefix[0]]]
  for t in range(num_chars+len(prefix)-1):
    #将上一时间步的输出作为当前时间步的输入
    X=to_onehot(tf.reshape(tf.constant([output[-1]]),shape=(1,1)),vocab_size)
    # print(X[0].shape)
    #计算输出和更新隐藏状态
    (Y,state)=rnn(X,state,params)
    #下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
    if t<len(prefix)-1:
      output.append(char_to_idx[prefix[t+1]])
    else:
      output.append(tf.argmax(Y[0],axis=1).numpy()[0])
  return ''.join([idx_to_char[i] for i in output])

我们先测试一下predict_rnn函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [0]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,idx_to_char,char_to_idx)

'分开爹奔漫条她黑长吴王近'

In [0]:
def data_iter_random(corpus_indices,batch_size,num_steps):
  #减1是因为输出的索引是相应输入的索引加1
  num_examples=(len(corpus_indices)-1)//num_steps
  epoch_size=num_examples//batch_size
  example_indices=list(range(num_examples))
  random.shuffle(example_indices)

  #返回从pos开始的长为num_steps的序列
  def _data(pos):
    return corpus_indices[pos:pos+num_steps]

  for i in range(epoch_size):
    #每次读取batch_size个随机样本
    i=i*batch_size
    batch_indices=example_indices[i:i+batch_size]
    X=[_data(j*num_steps) for j in batch_indices]
    Y=[_data(j*num_steps+1) for j in batch_indices]
    yield tf.constant(X),tf.constant(Y)

In [0]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps):
  corpus_indices=tf.constant(corpus_indices)
  data_len=len(corpus_indices)
  batch_len=data_len//batch_size
  indices=tf.reshape(corpus_indices[0:batch_size*batch_len],shape=(batch_size,batch_len))
  epoch_size=(batch_len-1) // num_steps
  for i in range(epoch_size):
    i=i*num_steps
    X=indices[:,i:i+num_steps]
    Y=indices[:,i+1:i+num_steps+1]
    yield X,Y

###6.4.5. 裁剪梯度
循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在“通过时间反向传播”一节中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量  $g $，并设裁剪的阈值是 $\theta$ 。裁剪后的梯度
$$
\min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}
$$
的 $L_2 $范数不超过 $\theta$ 。

In [0]:
def sgd(params,l,t,lr,batch_size,theta):
  norm=tf.constant([0],dtype=tf.float32)
  for param in params:
    dl_dp=t.gradient(l,param)
    norm+=tf.reduce_sum((dl_dp**2))
  norm=tf.sqrt(norm).numpy()
  if norm>theta:
    for param in params:
      dl_dp=t.gradient(l,param) #求梯度
      dl_dp=tf.assign(tf.Variable(dl_dp),dl_dp*theta/norm) #裁剪梯度
      param.assign_sub(lr*dl_dp/batch_size) #更新梯度

###6.4.6. 困惑度
我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下“softmax回归”一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。\
显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小vocab_size。

###6.4.7. 定义模型训练函数
跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

* 使用困惑度评价模型。
* 在迭代模型参数前裁剪梯度。
* 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考“语言模型数据集（周杰伦专辑歌词）”一节。
另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [0]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,is_random_iter,num_epochs,
                          num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
  if is_random_iter:
    data_iter_fn=data_iter_random
  else:
    data_iter_fn=data_iter_consecutive

  params=get_params()
  loss=losses.SparseCategoricalCrossentropy(from_logits=True)

  for epoch in range(num_epochs):
    if not is_random_iter:#如果使用相邻采样，在epoch开始时初始化隐藏状态
      state=init_rnn_state(batch_size,num_hiddens)
    l_sum,n,start=0.0,0,time.time()
    data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
    for X,Y in data_iter:
      if is_random_iter:#如果使用相邻采样，在每个小批量更新前初始化隐藏状态
        state=init_rnn_state(batch_size,num_hiddens)
      else:#否则需要使用detach函数从
        state=tf.stop_gradient(state) #停止计算该张量的梯度
      with tf.GradientTape(persistent=True) as t:
        t.watch(params)
        inputs=to_onehot(X,vocab_size)
        # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
        (outputs,state)=rnn(inputs,state,params)
        # 拼接之后形状为(num_steps * batch_size, vocab_size)
        outputs=tf.concat(values=[*outputs],axis=0)
        # Y的形状是(batch_size, num_steps)，转置后再变成长度为
        # batch * num_steps 的向量，这样跟输出的行一一对应
        y=tf.reshape(tf.transpose(Y),shape=(-1,))
        #使用交叉熵损失计算平均分类误差
        l=tf.reduce_mean(loss(y,outputs))
      sgd(params,l,t,lr,1,clipping_theta) #因为误差已经取过均值了,所以batch_size为1
      l_sum+=l.numpy()*y.numpy().shape[0]
      n+=y.numpy().shape[0]
    if(epoch +1)%10==0:
      print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, l_sum / n, time.time() - start))
      for prefix in prefixes:
          print(' -', predict_rnn(
              prefix, pred_len, rnn, params, init_rnn_state,
              num_hiddens, vocab_size, idx_to_char, char_to_idx))

###6.4.8. 训练模型并创作歌词
现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [0]:
train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,True,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 10, perplexity 5.707499, time 11.07 sec
 - 分开                                                  
 - 不分开                                                  
epoch 20, perplexity 5.434195, time 10.88 sec
 - 分开 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我
 - 不分开 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我 我
epoch 30, perplexity 5.042463, time 10.83 sec
 - 分开 我想 你不 我想你  我有你 我不 你想 我想你  我有你 我不 你想 我想你  我有你 我不 你
 - 不分开  哼 我有你 我不 你想你  我有你 我不 你想 我想你  我有你 我不 你想 我想你  我有你 
epoch 40, perplexity 4.638070, time 10.75 sec
 - 分开 我想要 不要的让我疯狂的可爱女人 温坏的让我疯狂的可爱女人 温坏的让我疯狂的可爱女人 温坏的让我疯
 - 不分开 我想的让我 我不能 不爱的让我疯狂的可爱女人 温坏的让我疯狂的可爱女人 温坏的让我疯狂的可爱女人 
epoch 50, perplexity 4.219021, time 10.73 sec
 - 分开 我想想这你 我不就 我不就的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让
 - 不分开 我想想 你爱我 我不就 我爱就的让爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
epoch 60, perplexity 3.818927, time 11.10 sec
 - 分开 我想要这生 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 不分开  一样人 你是我有多头 有知的美栈  哼什么我 别你的外 你已完起 我有一定热 有一定美  有在一
epoch 70, perplexity 3.414183, time 10.69 sec


接下来采用相邻采样训练模型并创作歌词。

In [0]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

###6.4.9. 小结
* 可以用基于字符级循环神经网络的语言模型来生成文本序列，例如创作歌词。
* 当训练循环神经网络时，为了应对梯度爆炸，可以裁剪梯度。
* 困惑度是对交叉熵损失函数做指数运算后得到的值。